<a href="https://colab.research.google.com/github/pengc7/DS5230/blob/main/DS5230_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data1 = pd.read_csv('US_counties_COVID19_health_weather_data.csv')
data1.shape

In [ ]:
#replace wrong value, encoding boolean values to float
data1['state']= data1['state'].replace(['M'],'Mississippi')
data1['presence_of_water_violation'] = data1['presence_of_water_violation'].astype(float)

In [ ]:
#remove weather variables and other irrelevant variables
data1.drop(data1.iloc[:,186:], axis=1, inplace=True)
data1.drop(data1.columns[[6,7,8,9]], axis=1, inplace=True)

#remove data from PR and VI, too many missing values
data1 = data1[~data1['fips'].isin(['PR', 'VI'])]
data1 = data1[~data1['state'].isin(['Alaska'])]

#some counties share the same fips (like 'PR' for counties in Porto Rico), so groupby state, county
data1sum = data1.groupby(['state', 'county'],as_index=False)[['cases','deaths']].sum()
data1mean = data1.groupby(['state', 'county'],as_index=False).mean()
data1mean.drop(['cases','deaths'],axis=1, inplace=True)

data = pd.merge(data1sum, data1mean, on=['state','county'],how='inner')

#get socioeconomic factors, impute missing values
factors = data.iloc[:,4:]
factors = factors.fillna(factors.mean())

normalization 

In [ ]:
#normalize
from sklearn.preprocessing import StandardScaler

# create a scaler object
std_scaler = StandardScaler()

# fit and transform the data
factors = std_scaler.fit_transform(factors)

Dimensionality reduction

In [ ]:
#standard PCA
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

X = factors
sk_pca = PCA(n_components=10)
pca_res = sk_pca.fit_transform(X)
pca_proj = sk_pca.inverse_transform(pca_res)
recon_error = np.linalg.norm((X-pca_proj),None)
print("Reconstruction_error: ", recon_error)
exp_var_pca = sk_pca.explained_variance_ratio_

plt.bar(range(0,len(exp_var_pca)), exp_var_pca, alpha=0.5, align='center')
plt.ylabel('Explained variance')
plt.xlabel('Principal component')
plt.show()

plt.scatter(pca_res[:, 0], pca_res[:, 1])
plt.xlabel('pc1')
plt.ylabel('pc2')
plt.show()

sum(exp_var_pca)

In [ ]:
#kernel PCA
#reference: https://stackoverflow.com/questions/53556359/selecting-kernel-and-hyperparameters-for-kernel-pca-reduction
from sklearn.decomposition import KernelPCA
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

X = factors

def my_scorer(estimator, X, y=None):
    X_reduced = estimator.transform(X)
    X_preimage = estimator.inverse_transform(X_reduced)
    return -1 * mean_squared_error(X, X_preimage)

kpca=KernelPCA(fit_inverse_transform=True, eigen_solver="randomized", random_state=100, alpha = 0.8)

#gridsearch
param_grid = [{
        "degree": [2,3],
        "gamma": np.linspace(1e-5, 5, 5),
        "kernel":["sigmoid","poly","rbf","sigmoid","cosine"]}]
grid_search = GridSearchCV(kpca, param_grid, cv=3, scoring=my_scorer)
grid_search.fit(X)
grid_search.best_params_

In [ ]:
X = factors
kpca = KernelPCA(kernel="cosine", eigen_solver="randomized",fit_inverse_transform=True, alpha=0.8, random_state=100)
kpca_res = kpca.fit_transform(X)
kpca_proj = kpca.inverse_transform(kpca_res)
kpca_recon_error = np.linalg.norm((X-kpca_proj),None)
print("Reconstruction_error: ", kpca_recon_error)

exp_var_kpca = np.var(kpca_res, axis=0)
exp_var_kpca_ratio = exp_var_kpca/np.sum(exp_var_kpca)
plt.bar(range(0,len(exp_var_kpca_ratio[:10])), exp_var_kpca_ratio[:10], alpha=0.5, align='center')
plt.ylabel('Explained variance')
plt.xlabel('Principal component')
plt.show()

plt.scatter(kpca_res[:, 0], kpca_res[:, 1])
plt.xlabel('pc1')
plt.ylabel('pc2')
plt.show()

sum(exp_var_kpca_ratio[:10])

In [ ]:
sum(exp_var_kpca_ratio[:20])

In [ ]:
#t-sne
#reference: https://plotly.com/python/t-sne-and-umap-projections/
from sklearn.manifold import TSNE
import plotly.express as px

X = kpca_res[:,:20]

tsne = TSNE(n_components=2, perplexity=30.0, verbose=1, random_state=100)
projections = tsne.fit_transform(X)

fig = px.scatter(
    projections, x=0, y=1,
)
fig.show()

In [ ]:
#install umap package
!pip install umap-learn

In [ ]:
#umap, reference https://plotly.com/python/t-sne-and-umap-projections/ ,https://umap-learn.readthedocs.io/en/latest/api.html
from umap import UMAP
import plotly.express as px

X= kpca_res[:,:20]
umap_2d = UMAP(n_components=2, n_neighbors=5, random_state=0)
proj_2d = umap_2d.fit_transform(X)
fig_2d = px.scatter(
    proj_2d, x=0, y=1,
)
fig_2d.show()

#umap_3d = UMAP(n_components=3, init='random', random_state=0)
#proj_3d = umap_3d.fit_transform(pca_res)
#fig_3d = px.scatter_3d(
#    proj_3d, x=0, y=1, z=2,
#)
#fig_3d.update_traces(marker_size=5)
#fig_3d.show()

Clustering

In [ ]:
#elbow choosing k
#reference: https://towardsdatascience.com/machine-learning-algorithms-part-9-k-means-example-in-python-f2ad05ed5203
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans

X = proj_2d
sse = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
    kmeans.fit(X)
    sse.append(kmeans.inertia_)
plt.plot(range(1, 11), sse)
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('SSE')
plt.show()

diff = [0]*(len(sse)-1)
for i in range(len(sse)-1):
  diff[i]=(sse[i+1]/sse[i])
#np.where(np.array(diff)<0.8)
diff

In [ ]:
from sklearn.metrics import silhouette_score

kmeans = KMeans(n_clusters=4, init='k-means++', max_iter=300, n_init=10, random_state=100)
labels = kmeans.fit_predict(X)
u_labels = np.unique(labels)

for i in u_labels:
    plt.scatter(X[labels == i , 0] , X[labels == i , 1] , label = i)
plt.legend()
plt.show()

score=silhouette_score(X, labels)

print("silhouette score", score)

In [ ]:
#spectral clustering
from sklearn.cluster import SpectralClustering
 
X = proj_2d

specCluster = SpectralClustering(n_clusters=4,random_state=200).fit(X)
slabels = specCluster.labels_
s_labels = np.unique(slabels)
for i in s_labels:
    plt.scatter(X[slabels == i , 0] , X[slabels == i , 1] , label = i)
plt.legend()
plt.show()

sil_score = silhouette_score(X, specCluster.labels_)
print("silhouette score : %f"%(sil_score))

Classification

In [ ]:
#transform to df
df = pd.DataFrame(factors,index=data.iloc[:,4:180].index, columns=data.iloc[:,4:180].columns )

In [ ]:
#xgboost
#reference: https://www.datacamp.com/community/tutorials/xgboost-in-python
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import plot_importance
from sklearn.metrics import roc_curve,auc,roc_auc_score

X,y=df,specCluster.labels_
data_dmatrix = xgb.DMatrix(data=factors,label=specCluster.labels_)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
xg_classifier = xgb.XGBClassifier(objective ='multi:softmax', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)#,
xg_classifier.fit(X_train,y_train)
preds = xg_classifier.predict(X_test)
rmse = accuracy_score(y_test, preds)
print(rmse)

# plot feature importance
plot_importance(xg_classifier,max_num_features=10)
plt.show()

In [ ]:
#cluster the counties
data['cluster']=pd.Series(specCluster.labels_)
clstr = data.groupby(['cluster'],as_index=False)[['cases','deaths','percent_fair_or_poor_health',
                                        "percent_smokers","population_density_per_sqmi"]].mean()
clstr['fatality'] = clstr['deaths']/clstr['cases'] 
clstr[['cases','deaths','fatality']]                                 

In [ ]:
clstr['population_density_per_sqmi']

In [ ]:
Xx1 = "percent_fair_or_poor_health"
boxplot1 = data.boxplot(column=x1,by="cluster")
plt.ylabel(x1)
plt.title(" ")
plt.show()

In [ ]:
x2 = "average_daily_pm2_5"
boxplot2 = data.boxplot(column=x2,by="cluster")
plt.ylabel(x2)
plt.title(" ")
plt.show()

In [ ]:
x3 = "population_density_per_sqmi"
boxplot3 = data.boxplot(column=x3,by="cluster")
plt.ylabel(x3)
plt.title(" ")
plt.show()

In [ ]:
geo1 = data[['state','county','cluster']]
geo2 = data1[['state', 'county','fips']].drop_duplicates()
geo = pd.merge(geo1, geo2, on = ['state','county'])
geo['cluster'] = geo['cluster'].apply(str)

In [ ]:
#reference: https://plotly.com/python
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

import plotly.express as px

fig = px.choropleth_mapbox(geo, geojson=counties, locations='fips', color='cluster',
                           mapbox_style="carto-positron",
                           zoom=3, center = {"lat": 37.0902, "lon": -95.7129},
                           labels={'cluster':'cluster'},
                            category_orders={"cluster": ["0","1","2","3","4"]}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()